In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

scala_version='2.12'
spark_version='3.5.1'

package = f"org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version},org.apache.kafka:kafka-clients:3.5.0"


spark = SparkSession.builder \
    .appName("kafka-example") \
    .config("spark.jars.packages", package) \
    .config("spark.ui.showConsoleProgress", "false") \
    .getOrCreate()

spark

In [9]:
kafka_server = "kafka:9092" 
topic_name = "LeNguyenHoangPhuc_RandomNumber"         

streamRawDf = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", kafka_server) \
    .option("subscribe", topic_name) \
    .load()


In [10]:
from pyspark.sql.functions import col
streamDF = streamRawDf.select(col('topic'),col('offset'),col('value').cast('string').substr(12,1).alias('rand_number'))
checkEvenDF = streamDF.withColumn('Is_Even',col('rand_number').cast('int') % 2 == 0 )

In [11]:
from random import randint
randNum=str(randint(0,10000))
q1name = "queryNumber"+randNum
q2name = "queryCheckEven"+randNum
stream_writer1 = (streamDF.writeStream.queryName(q1name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
stream_writer2 = (checkEvenDF.writeStream.queryName(q2name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
query1 = stream_writer1.start()
query2 = stream_writer2.start()

In [ ]:
from time import sleep
from IPython.display import display, clear_output

x = 0
try:
    while(True):
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        result1 = spark.sql(f"select * from {query1.name}")
        result2 = spark.sql(f"select * from {query2.name}")
        display(result1.toPandas())
        display(result2.toPandas())
        sleep(5)
        x += 1
        clear_output(wait=True)
except KeyboardInterrupt:
    print("break")
print("Live view ended")

Showing live view refreshed every 5 seconds
Seconds passed: 225


,topic,offset,rand_number
0,LeNguyenHoangPhuc_RandomNumber,49,4
1,LeNguyenHoangPhuc_RandomNumber,50,5
2,LeNguyenHoangPhuc_RandomNumber,51,5
3,LeNguyenHoangPhuc_RandomNumber,52,5
4,LeNguyenHoangPhuc_RandomNumber,53,5
...,...,...,...
169,LeNguyenHoangPhuc_RandomNumber,218,5
170,LeNguyenHoangPhuc_RandomNumber,219,5
171,LeNguyenHoangPhuc_RandomNumber,220,5
172,LeNguyenHoangPhuc_RandomNumber,221,5


,topic,offset,rand_number,Is_Even
0,LeNguyenHoangPhuc_RandomNumber,49,4,True
1,LeNguyenHoangPhuc_RandomNumber,50,5,False
2,LeNguyenHoangPhuc_RandomNumber,51,5,False
3,LeNguyenHoangPhuc_RandomNumber,52,5,False
4,LeNguyenHoangPhuc_RandomNumber,53,5,False
...,...,...,...,...
169,LeNguyenHoangPhuc_RandomNumber,218,5,False
170,LeNguyenHoangPhuc_RandomNumber,219,5,False
171,LeNguyenHoangPhuc_RandomNumber,220,5,False
172,LeNguyenHoangPhuc_RandomNumber,221,5,False


break
Live view ended
